In [4]:
import rasterio
import yaml
import os
import geopandas as gpd
import pandas as pd
from osgeo import gdal
from pathlib import Path
import sys
import cv2 as cv

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import execution.orchestrate_funcs as orchestrate_funcs





In [2]:

config_file = '/fs/ess/PAS2699/nitrogen/data/uas/2023/config/uas_config.yaml'

# Load and process configuration
config = om_funcs.load_config(config_file)

base_folder = config['base_folder']
print(f'base folder: {base_folder}')

flight_folder = os.path.join(base_folder, 'flights')

om_aligned_folder = os.path.join(base_folder, config['om_aligned_folder'])
print(f'om folder: {om_aligned_folder}')

# Extract flight configuration dictionary
flight_config_dict = config['flight_list']

# Extract sensor_dict
sensor_dict = config['sensor_dict']

base folder: /fs/ess/PAS2699/nitrogen/data/uas/2023/
om folder: /fs/ess/PAS2699/nitrogen/data/uas/2023/orthomosaics/aligned/


In [3]:
# Adjust the transform of a set of orthomosaics

def om_nudge(location, sensor,  om_aligned_folder, x_adj, y_adj):

    for file in os.listdir(om_aligned_folder):
        if file.endswith('aligned.tif') and location in file and sensor in file:
            print(f'Processing {file}')
            om_file = os.path.join(om_aligned_folder, file)
            print(f'om_file: {om_file}')
            with rasterio.open(om_file) as src:
                transform = src.transform
                data = src.read()
                meta = src.meta.copy()

            new_transform = rasterio.Affine(transform.a, transform.b, transform.c + x_adj, transform.d, transform.e, transform.f + y_adj)
            meta.update({'transform': new_transform,
                         'compress': 'deflate'})

            base, ext = os.path.splitext(om_file)
            new_file = f"{base}_nudged{ext}"

            with rasterio.open(new_file, 'w', **meta) as dst:
                    dst.write(data)
            print(f'Wrote new nudged file: {new_file}')

om_nudge('western_n', 'ms', om_aligned_folder, .3, -3.42)


Processing western_n_ms_20240801_aligned.tif
om_file: /fs/ess/PAS2699/nitrogen/data/uas/2024/orthomosaics/v2_5_aligned/western_n_ms_20240801_aligned.tif
Wrote new nudged file: /fs/ess/PAS2699/nitrogen/data/uas/2024/orthomosaics/v2_5_aligned/western_n_ms_20240801_aligned.tif_nudged.tif
Processing western_n_ms_20240621_aligned.tif
om_file: /fs/ess/PAS2699/nitrogen/data/uas/2024/orthomosaics/v2_5_aligned/western_n_ms_20240621_aligned.tif
Wrote new nudged file: /fs/ess/PAS2699/nitrogen/data/uas/2024/orthomosaics/v2_5_aligned/western_n_ms_20240621_aligned.tif_nudged.tif
Processing western_n_ms_20240724_aligned.tif
om_file: /fs/ess/PAS2699/nitrogen/data/uas/2024/orthomosaics/v2_5_aligned/western_n_ms_20240724_aligned.tif
Wrote new nudged file: /fs/ess/PAS2699/nitrogen/data/uas/2024/orthomosaics/v2_5_aligned/western_n_ms_20240724_aligned.tif_nudged.tif
Processing western_n_ms_20240613_aligned.tif
om_file: /fs/ess/PAS2699/nitrogen/data/uas/2024/orthomosaics/v2_5_aligned/western_n_ms_20240613_a

In [14]:
# This function is a more efficient way to adjust the geotransform of a raster using GDAL without having to rewrite the entire raster.

def om_nudge_in_place(location, sensor, om_aligned_folder, x_adj, y_adj, date = None):
    for file in os.listdir(om_aligned_folder):
        if file.endswith('aligned.tif') and location in file and sensor in file:
            # If a date is provided, skip files that don't include the date.
            if date is not None and date not in file:
                continue
            print(f'Processing {file}')
            om_file = os.path.join(om_aligned_folder, file)
            ds = gdal.Open(om_file, gdal.GA_Update)
            if ds is None:
                print(f"Failed to open {om_file} in update mode")
                continue
            
            # Get the current geotransform and update the x and y origin values.
            geotransform = ds.GetGeoTransform()
            new_geotransform = (
                geotransform[0] + x_adj,  # Adjusted top left x
                geotransform[1],          # Pixel width remains the same
                geotransform[2],          # Rotation (if any) remains the same
                geotransform[3] + y_adj,  # Adjusted top left y
                geotransform[4],          # Rotation (if any) remains the same
                geotransform[5]           # Pixel height remains the same
            )
            
            ds.SetGeoTransform(new_geotransform)
            ds = None  # Closing the dataset writes the changes
            print(f'Updated geotransform for {om_file}')

# Adjusts
om_nudge_in_place('northwest', 'ms', om_aligned_folder, -.1, 0)




Processing northwest_ms_20230531_aligned.tif
Updated geotransform for /fs/ess/PAS2699/nitrogen/data/uas/2023/orthomosaics/aligned/northwest_ms_20230531_aligned.tif
Processing northwest_ms_20230606_aligned.tif
Updated geotransform for /fs/ess/PAS2699/nitrogen/data/uas/2023/orthomosaics/aligned/northwest_ms_20230606_aligned.tif
Processing northwest_ms_20230525_aligned.tif
Updated geotransform for /fs/ess/PAS2699/nitrogen/data/uas/2023/orthomosaics/aligned/northwest_ms_20230525_aligned.tif
Processing northwest_ms_20230613_aligned.tif
Updated geotransform for /fs/ess/PAS2699/nitrogen/data/uas/2023/orthomosaics/aligned/northwest_ms_20230613_aligned.tif
Processing northwest_ms_20230621_aligned.tif
Updated geotransform for /fs/ess/PAS2699/nitrogen/data/uas/2023/orthomosaics/aligned/northwest_ms_20230621_aligned.tif


# Nudge history

om_nudge('wooster_n', 'ms', om_aligned_folder, 2.05, -0.45)

In [12]:
# Shapefile nudge




def shapefile_nudge(shapefile_path, x_adj, y_adj):
    gdf_plots = gpd.read_file(shapefile_path)
    #convert to utm zone 17N
    gdf_plots = gdf_plots.to_crs(epsg=32617)

    gdf_plots['geometry'] = gdf_plots['geometry'].translate(x_adj, y_adj)    
    gdf_plots.to_file(shapefile_path)


shapefile_path = '/fs/ess/PAS2699/nitrogen/data/uas/2024/shapefiles/northwest_bftb_soy/northwest_bftb_soy.shp'
shapefile_nudge(shapefile_path, -.68, 0)


In [2]:
folder = '/fs/ess/PAS2699/nitrogen/data/uas/2025/orthomosaics/aligned_staging'

for file in os.listdir(folder):
    if file.endswith('.tif'):
        print(file)
        om_file = os.path.join(folder, file)
        print(om_file)
        with rasterio.open(om_file) as src:
            transform = src.transform
            print(f'transform: {transform}')
            

frantom_b89east_rgb_20250617_aligned_parallel.tif
/fs/ess/PAS2699/nitrogen/data/uas/2025/orthomosaics/aligned_staging/frantom_b89east_rgb_20250617_aligned_parallel.tif
transform: | 0.01, 0.00, 738466.91|
| 0.00,-0.01, 4452649.22|
| 0.00, 0.00, 1.00|
frantom_b89east_rgb_20250617_aligned_new.tif
/fs/ess/PAS2699/nitrogen/data/uas/2025/orthomosaics/aligned_staging/frantom_b89east_rgb_20250617_aligned_new.tif
transform: | 0.01, 0.00, 738461.62|
| 0.00,-0.01, 4452646.62|
| 0.00, 0.00, 1.00|
frantom_b89east_rgb_20250617_aligned_new2.tif
/fs/ess/PAS2699/nitrogen/data/uas/2025/orthomosaics/aligned_staging/frantom_b89east_rgb_20250617_aligned_new2.tif
transform: | 0.01, 0.00, 738461.62|
| 0.00,-0.01, 4452646.62|
| 0.00, 0.00, 1.00|
frantom_b89east_rgb_20250617_aligned_old.tif
/fs/ess/PAS2699/nitrogen/data/uas/2025/orthomosaics/aligned_staging/frantom_b89east_rgb_20250617_aligned_old.tif
transform: | 0.01, 0.00, 738461.71|
| 0.00,-0.01, 4452646.55|
| 0.00, 0.00, 1.00|
frantom_b89east_rgb_20250617

In [13]:
new_images_path = '/fs/ess/PAS2699/nitrogen/data/uas/2025/orthomosaics/aligned/debug_new'

for i, file in enumerate(sorted(os.listdir(new_images_path))):
    if file.endswith('.png'):
        print(file)
        new_file = os.path.join(new_images_path, file)
        new_image = cv.imread(new_file)
        print(f'             new_file shape: {new_image.shape}')



best_match_patch_0.png
             new_file shape: (34, 34, 3)
best_match_patch_1.png
             new_file shape: (25, 34, 3)
best_match_patch_2.png
             new_file shape: (34, 34, 3)
best_match_patch_3.png
             new_file shape: (34, 34, 3)
best_match_patch_4.png
             new_file shape: (34, 34, 3)
best_match_patch_5.png
             new_file shape: (24, 34, 3)
best_match_patch_6.png
             new_file shape: (17, 34, 3)
best_match_patch_7.png
             new_file shape: (34, 34, 3)
best_match_template_0.png
             new_file shape: (34, 34, 3)
best_match_template_1.png
             new_file shape: (25, 34, 3)
best_match_template_2.png
             new_file shape: (34, 34, 3)
best_match_template_3.png
             new_file shape: (34, 34, 3)
best_match_template_4.png
             new_file shape: (34, 34, 3)
best_match_template_5.png
             new_file shape: (24, 34, 3)
best_match_template_6.png
             new_file shape: (17, 34, 3)
best_match_template

In [15]:
old_images_path = '/fs/ess/PAS2699/nitrogen/data/uas/2025/orthomosaics/aligned/debug_old'

for i, file in enumerate(sorted(os.listdir(old_images_path))):
    if file.endswith('.png'):
        print(file)
        old_file = os.path.join(old_images_path, file)
        old_image = cv.imread(old_file)
        print(f'             new_file shape: {old_image.shape}')

search_0.png
             new_file shape: (171, 171, 3)
search_1.png
             new_file shape: (139, 171, 3)
search_2.png
             new_file shape: (171, 171, 3)
search_3.png
             new_file shape: (171, 171, 3)
search_4.png
             new_file shape: (171, 171, 3)
search_5.png
             new_file shape: (171, 171, 3)
search_6.png
             new_file shape: (171, 171, 3)
template_1.png
             new_file shape: (35, 35, 3)
template_10.png
             new_file shape: (35, 35, 3)
template_11.png
             new_file shape: (35, 35, 3)
template_12.png
             new_file shape: (35, 35, 3)
template_13.png
             new_file shape: (25, 35, 3)
template_14.png
             new_file shape: (17, 35, 3)
template_15.png
             new_file shape: (35, 35, 3)
template_2.png
             new_file shape: (25, 35, 3)
template_3.png
             new_file shape: (35, 35, 3)
template_4.png
             new_file shape: (35, 35, 3)
template_5.png
             new_file shape

In [ ]:

shp_file = ''